# Are pathogenic mutations more likely to have modifications nearby?
In `studyBias_SwissProt.ipynb` we explored the relationship between annotations, PTMs, and mutations.  We found that there is a clear study bias for proteins that contain pathogenic mutations, having more GO terms, PTMs, and mutations.  Therefore, here we will consider only mutations coming from these heavily studied proteins as we explore the relationship between mutations and nearby PTMs.

In [1]:
# Setup the workspace, 
from proteomeScoutAPI import ProteomeScoutAPI
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from pylab import *
import pandas as pd
from scipy import stats 
import pickle
import random


[df_path, protID_path] = pickle.load(open("pathoProteins.p", "r"))
%matplotlib inline
#proteomeScoutFile = '/Users/knaegle/HG/knaegle_repo/mutationsAnalysis/data/proteomescout_mammalia_20150201.tsv'
proteomeScoutFile = '/Users/alex/Dropbox/WASHU/naegle/mutants_paper/data/v1.0a_July_2015/proteomescout_everything_20150712.tsv'

# read in ProteomeScout data
PTM_API = ProteomeScoutAPI(proteomeScoutFile)

# read in SwissProt mutations data 
SwissProtMuts = pickle.load(open('swissprot_mutations.p','rb'))

modWindow = 8 #change this to 
pValCutoff = 0.05

df_path.describe()



,GO,Mutations,PTMs,Sequence
count,2273.000000,2273.000000,2273.000000,2273.000000
mean,16.529696,16.033436,19.881654,758.413550
std,15.319316,27.438130,25.970961,729.093742
min,0.000000,1.000000,1.000000,51.000000
25%,7.000000,3.000000,5.000000,367.000000
50%,12.000000,7.000000,12.000000,532.000000
75%,21.000000,16.000000,24.000000,865.000000
max,159.000000,275.000000,316.000000,8797.000000


## Build mutations dataset
The following code moves through each protein we identified previously as having one or more pathogenic mutations, and for each mutations determines 
1. If it's a pathogenic mutation or not, and 
2. If it lies within a `modWindow` distance of a PTM, as well what that PTM is (assuming its one of phosphoserine, phosphothreonine, phosphotyrosine, ubiquitination, or 6-acetylylysine)

In [2]:
#### Create an object of all mutations that exist on proteins that have at least one pathogenic mutation.
# Also, add the number of nearby modifications of the major types, where nearby is ste on +/-7 amino acids
d = pd.DataFrame(columns=['ID', 'mod_pos', 'amino acid', 'pathoBit', 'Phosphoserine', 'Phosphothreonine', 'Phosphotyrosine',
'N6-acetyllysine', 'Ubiquitination'])

# for each protein
for ID in protID_path:
    
    # if we have mutation data
    if ID in SwissProtMuts:
        mutations = SwissProtMuts[ID]
 
        # for each mutation associated with protine $ID
        for mut in mutations:
            pos, from_res, to_res, patho_status = mut
        
            # get all PTMs which lie in a modWindow region (+/-) around
            # the mutation site
            mods = PTM_API.get_nearbyPTMs(ID, int(pos), modWindow)
        
            # if the mutation is associated with disease set tha pathoBit to 1,
            # else remains 0
            pathoBit = 0
            if patho_status == 'Disease':
                pathoBit = 1
    
            # create an initial protein record for the mutation, setting all 
            # modification counters to zero
            temp = pd.Series({'ID': ID, 'mod_pos':pos, 'amino acid':from_res, 'pathoBit':pathoBit, 
                            'Phosphoserine':0, 'Phosphothreonine':0, 'Phosphotyrosine':0, 'N6-acetyllysine':0, 
                            'Ubiquitination':0})
                    
            # for any modifications near the mutation (as defined by modWindow) increment
            # the appropriate modification counter
            for mod in mods:
                mod_pos, aa, mod_type = mod          
                try:
                    temp[mod_type] +=1
                except:
                    # we only care about 5 types of modification (others are to rare) so ignore
                    # other types
                    pass
                
            # finally append this record to the dataframe
            d = d.append(temp, ignore_index='True') 
    else:
        # no mutations, skip this protein!
        pass
        
d.sum()

ID                  P30613P30613P30613P30613P30613P30613P30613P306...
mod_pos                                                  1.956672e+07
amino acid          MVLERGDFIDGARRDGIRRNVTANNTQRRAGAVARRARRRRRVRNG...
pathoBit                                                        25688
Phosphoserine                                                    5531
Phosphothreonine                                                 2831
Phosphotyrosine                                                  3007
N6-acetyllysine                                                  2038
Ubiquitination                                                   2086
dtype: object

### Checking for enrichment of mods near pathogenic mutations
Here is a description of the mutations and nearby modifications that make up the dataset we will consider, based on those annotations that come from proteins that have at least one pathogenic mutation.  We will use the Fisher Exact test to determine if there are enrichment differences. 

In [3]:
# Select only unique mutations based on protein and position.  Prioritize pathogenicity and binarize nearby mods 
dCollapse = pd.DataFrame(columns=d.columns)

for uniqueKey, row in d.groupby(['ID', 'mod_pos', 'amino acid']):
    temp = pd.DataFrame([[uniqueKey[0], uniqueKey[1], uniqueKey[2], int(row['pathoBit'].sum() > 0), 
                         int(row['Phosphoserine'].sum() > 0), int(row['Phosphothreonine'].sum() > 0), 
                        int(row['Phosphotyrosine'].sum() > 0), int(row['N6-acetyllysine'].sum() > 0), 
                       int(row['Ubiquitination'].sum() > 0)]], columns=dCollapse.columns)
    dCollapse = dCollapse.append(temp)
dCollapse.describe()

,mod_pos,pathoBit,Phosphoserine,Phosphothreonine,Phosphotyrosine,N6-acetyllysine,Ubiquitination
count,32094.000000,32094.000000,32094.000000,32094.000000,32094.000000,32094.000000,32094.000000
mean,551.864928,0.685455,0.106219,0.063532,0.067240,0.040724,0.047018
std,699.379522,0.464342,0.308123,0.243921,0.250441,0.197653,0.211681
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,158.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,328.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,641.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8741.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
def FDR(pArr, alpha):
    """
    False Discovery Rate (FDR)
    Returns a Q value for significant values in pArr for a target FDR of alpha
    
    pArr  : Array of p-values
    alpha : Confidence threshold
    """
    
    # number of p-values
    m = len(pArr)
    
    # sort the p-values (smallest first)
    pValSort = sorted(pArr, key=lambda x: float(x))
        
    # assuming there are no rejected hypotheses at all
    pAdj = 0; 
    
    # cycle through each p-value
    for i in range (0, len(pValSort)):
        pi = pValSort[i]
        j = i+1;
        # print "Comparing %0.E to %0.2f"%(pi, (j*alpha/m))
        if(pi <= (j*alpha/m)):
            pAdj = pi
        else:
            # if the p-value is greater use the previously set
            # adjusted p-value
            return pAdj

In [5]:
def printPathoModStats(d):
    """
    Check for enrichment of pathogenic mutations near sites of modifications
    
    d :  binarized data frame defined above
    """

    # get number of mutation sites    
    N = len(d['pathoBit'])
    
    # get the number of mutation sites associated with disease    
    K = d['pathoBit'].sum()
    
    pvalueArr = []
    strSig = ''
    alpha = 0.05
    
    # for each of the modification types
    for mods in d.columns[-5:]:
        
        # number of mutation sites which have a modification 
        # of type $mods nearby
        n = len(d[d[mods]==1])

        # number of disease-related mutation which have a modification 
        # of type $mods nearby
        k = len(d[(d['pathoBit']==1) & (d[mods]==1)])
        
        # use the Fisher's exact test to take the following 
        # contigency table 
        # 
        #  
        #                        PATHOGENIC   NON-PATHOGENIC
        #  MUTATIONS NEAR SITE |   k              n-k
        #  MUTATIONS ANYWHERE  |   K              N-K
        #
        # And ask if the difference between the observations might occur by chance - the p
        # value reports on the likelyhood of finding pathogenic mutations near a site and non
        # pathogenic mutants away from a site occuring by chance vs. what we see - lower means
        # more sure it's a real effect
        oddsratio, pvalue = stats.fisher_exact([[k, n-k], [K, N-K]], alternative='greater')
        pvalueArr.append(pvalue)
        strSig = ''
        
        # for examples where the p-value is less than the defined alpha append a *
        if pvalue <= alpha:
            strSig = '*' 
        print "%2s %17s:\t %2.E \t(N=%d, n=%d, K=%d, k=%d)"%(strSig, mods, pvalue, N, n, K, k)
        
    # use the false discovery rate calculate the adjusted p-value
    p_adjust = FDR(pvalueArr, alpha)
    print "adjusted p-value is %0.3f"%(p_adjust)

        
    


In [6]:
printPathoModStats(dCollapse)

       Phosphoserine:	 1E+00 	(N=32094, n=3409, K=21999, k=2100)
    Phosphothreonine:	 1E+00 	(N=32094, n=2039, K=21999, k=1241)
 *   Phosphotyrosine:	 2E-05 	(N=32094, n=2158, K=21999, k=1569)
     N6-acetyllysine:	 1E-01 	(N=32094, n=1307, K=21999, k=915)
 *    Ubiquitination:	 2E-03 	(N=32094, n=1509, K=21999, k=1088)
adjusted p-value is 0.002


# What is the distribution of amino acid types
Is there anything significant about the type of amino acids mutated and associated with pathogenicity?

In [7]:
# for each unique amino acid calculate whether there is over or under-representation (two-tailed test)

# Get the number of mutation sites and the number of pathogenic mutations (N and K respectively)
N = len(dCollapse['pathoBit'])
K = dCollapse['pathoBit'].sum()

runningSumCheck = 0
aaDict = {}

# cycle through each amino acid, collecting all the sites where
# that amino acid is mutated (row)
for uniqueKey, row in dCollapse.groupby(['amino acid']):
    
    # get current amino acids and number of mutation sites on that amino acid
    aa = uniqueKey[0]
    n = len(dCollapse[dCollapse['amino acid']==aa]) # aka len(row)
    
    # save the number of times that amino acid is observed on a mutation site
    aaDict[aa] = n
    
    # increment for book keeping
    runningSumCheck += n
    
    # get the number of occurences where a mutation to the $aa amino acid represents
    # a pathogenic mutation
    k = len(dCollapse[(dCollapse['amino acid']==aa) & (dCollapse['pathoBit']==1)])
    
    # use the Fisher's exact test to take the following 
    # contigency table 
    # 
    #  
    #                        PATHOGENIC   NON-PATHOGENIC
    #  MUTATIONS ON AA $aa |   k              n-k
    #  MUTATIONS ANYWHERE  |   K              N-K
    #
    # Smaller numbers mean we preferentially see pathogenic mutations on
    # some amino acid and less frequently see mutations on that amino
    # acid which are not pathogenic
    #    
    oddsratio, pvalue = stats.fisher_exact([[k, n-k], [K, N-K]])

    # Bonferroni correction, alpha at 0.05
    if pvalue <= pValCutoff/20: 
        print "%3s:\t %0.E \t(N=%d, n=%d, K=%d, k=%d)"%(aa, pvalue, N, n, K, k)
        
if runningSumCheck != N:
    print "Error in Parity: Sum is %d instead of %d"%(runningSumCheck, N)
    



  A:	 1E-16 	(N=32094, n=2304, K=21999, k=1384)
  C:	 2E-66 	(N=32094, n=1318, K=21999, k=1173)
  E:	 1E-04 	(N=32094, n=1531, K=21999, k=976)
  F:	 3E-05 	(N=32094, n=797, K=21999, k=601)
  G:	 4E-45 	(N=32094, n=3413, K=21999, k=2725)
  I:	 2E-07 	(N=32094, n=1240, K=21999, k=761)
  K:	 2E-09 	(N=32094, n=813, K=21999, k=475)
  L:	 7E-12 	(N=32094, n=2324, K=21999, k=1749)
  M:	 4E-04 	(N=32094, n=793, K=21999, k=496)
  N:	 4E-04 	(N=32094, n=1024, K=21999, k=648)
  P:	 1E-06 	(N=32094, n=1865, K=21999, k=1177)
  Q:	 5E-07 	(N=32094, n=805, K=21999, k=483)
  R:	 1E-04 	(N=32094, n=4746, K=21999, k=3384)
  S:	 9E-04 	(N=32094, n=1911, K=21999, k=1240)
  T:	 3E-18 	(N=32094, n=1617, K=21999, k=937)
  V:	 6E-32 	(N=32094, n=1852, K=21999, k=1020)
  W:	 5E-22 	(N=32094, n=520, K=21999, k=452)
  Y:	 5E-17 	(N=32094, n=908, K=21999, k=736)


## Examine random site-mutation enrichment
We checked to see if there was enrichment in nearby PTMs for any randomly selected mutation set based on selecting for the same distribution of amino acids. In other words, taking the same data set by randomly reshuffling which mutations were pathogenic and which were not, do we find a specific enrichment of pathogenic mutations near sites?

To do this, we create a new set based on selecting random permutations of the lsits that define particular amino acid types rebuild a fake patho set that is defined based on distribution of amino acids then check for enrichment of nearby mods


In [8]:
numRepeats = 10
print "Printing observed enrichments for random sets. Remember expected rate of seeing a p-value <=%0.2f is %1.2f times"%(pValCutoff, pValCutoff*5*numRepeats)
for i in range(1,numRepeats):
    
    # create an empty dataframe using the original column names
    dS = pd.DataFrame(columns=dCollapse.columns)
    
    # for each different amino acid
    for uniqueKey, row in dCollapse.groupby(['amino acid']):
        aa = uniqueKey[0]
        
        # this is just row (..)
        aaMuts = dCollapse[dCollapse['amino acid']==aa]

        # totally randomize the order of each of the mutation sites
        # associated with mutations of residue aa
        aaShuffled = aaMuts.iloc[np.random.permutation(aaDict[aa])]
        
        # have to explicity set index values so we can assign a subsection
        aaShuffled.index = range(0,len(aaShuffled))

        # Determine the number of pathogenic mutations associated with sites
        num = len(dCollapse[(dCollapse['amino acid']==aa) & (dCollapse['pathoBit']==1)])
        
        # set the first $num sites to be pathogenic and the rest to be not
        aaShuffled.loc[0:num, 'pathoBit'] = 1
        aaShuffled.loc[num:len(aaMuts), 'pathoBit'] = 0
        
        # sanity check!
        if num != aaShuffled['pathoBit'].sum():
            print "Problem for %s: wanted %d and retrieved %d"%(aa, num, aaShuffled['pathoBit'].sum())
            print "\t number of amino acids in set is %d and I have length of set %d"%(aaDict[aa], len(aaShuffled))
            
        # finally, append this set of data to the synthetic dataframe being created!
        dS = dS.append(aaShuffled)
    # now print enrichment
    print "Run %d"%(i)
    printPathoModStats(dS)

Printing observed enrichments for random sets. Remember expected rate of seeing a p-value <=0.05 is 2.50 times
Run 1
       Phosphoserine:	 9E-01 	(N=32094, n=3409, K=21999, k=2291)
    Phosphothreonine:	 1E+00 	(N=32094, n=2039, K=21999, k=1349)
     Phosphotyrosine:	 7E-01 	(N=32094, n=2158, K=21999, k=1467)
     N6-acetyllysine:	 5E-01 	(N=32094, n=1307, K=21999, k=895)
      Ubiquitination:	 8E-01 	(N=32094, n=1509, K=21999, k=1022)
adjusted p-value is 0.000
Run 2
       Phosphoserine:	 9E-01 	(N=32094, n=3409, K=21999, k=2307)
    Phosphothreonine:	 8E-02 	(N=32094, n=2039, K=21999, k=1429)
     Phosphotyrosine:	 6E-02 	(N=32094, n=2158, K=21999, k=1514)
     N6-acetyllysine:	 3E-01 	(N=32094, n=1307, K=21999, k=907)
      Ubiquitination:	 3E-01 	(N=32094, n=1509, K=21999, k=1044)
adjusted p-value is 0.000
Run 3
       Phosphoserine:	 8E-01 	(N=32094, n=3409, K=21999, k=2308)
    Phosphothreonine:	 3E-01 	(N=32094, n=2039, K=21999, k=1407)
     Phosphotyrosine:	 3E-01 	(N=32094, n